# Data Mining Course Spark Exercise
## Sharif University of Technology

In this notebook we are going to analyze farsi news.
Outline of the exercise:
* Dataset preparation
* Preprocessing
* Exploration
* Word Collections

You should replace the `TODO` parts with your implementation. Remeber that each `TODO` may take multiple lines and you shouldn't limit your self to one-line codes.

## Prerequisites
You should be faimilar with [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf). In this notebook you should use the following formula for tf-idf:
$$f_{t,d}/len(d) \times log(1 + \frac{N}{n_t})$$

## Warning: RDD api only
You **can not** use Dataframe, Dataset, mllib, ml, ... apis of spark in this exercise. You should only use the [RDD api](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD).

# Please enter your name below:
# Name: Mohammad Javad Noroozi
# Student Number: 99102434

# HW2

## Install Pypark & Initialization
Uncomment this section if you use google colab or local pc

In [ ]:
# ! pip install pyspark

In [1]:
import findspark
findspark.find()
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("HW1") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

23/12/31 19:56:32 WARN Utils: Your hostname, javad-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 172.27.169.255 instead (on interface wlp0s20f3)
23/12/31 19:56:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/31 19:56:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Reading the data

In [2]:
news_rdd = sc.textFile("news_data.jsonl")
news_rdd.takeSample(False, 1, 12)

['{"body": "نشست بزرگداشت روز جهانی حافظ شیرازی 20 مهرماه در دانشگاه پکن برگزار می شود.\\n\\n- ایرنا نوشت: با همکاری مشترک گروه زبان و ادبیات فارسی دانشگاه پکن و رایزنی فرهنگی جمهوری اسلامی ایران در چین، نشست بزرگداشت روز جهانی حافظ شیرازی روز پنجشنبه 20 مهرماه ساعت 10:30 تا 12 در دانشگاه پکن برگزار می شود.\\n\\nدر این نشست، محسن بختیار، سفیر و نعمت الله ایران زاده، رایزن فرهنگی جمهوری اسلامی ایران در چین، پروفسور چن مینْگ، رییس محترم دانشکده زبان های خارجی دانشگاه پکن، بهادر باقری، استاد مدعو زبان و ادبیات فارسی دانشکده زبان های خارجی دانشگاه پکن، سخنرانی می کنند.\\n\\nخانم وانگ یی دان، استاد رشته زبان و ادبیات فارسی دانشگاه پکن، شی گوانگ، استاد رشته زبان و ادبیات فارسی و مدیر مرکز پژوهش های فرهنگ ایران در دانشگاه پکن و لیو یینگ جون، استاد و مدیر گروه زبان و ادبیات فارسی دانشگاه پکن از دیگر سخنرانان این نشست هستند.", "image_title_url": "https://ettelaat.com/files/fa/news/1402/7/20/82679_282.jpg", "language": "fa", "source": "موتور جستجوی قطره", "title": "ادای احترام چینی ها به حافظ +ع

### import neccesary packages

In [3]:
import json
import re
import math
import random
import datetime
import numpy as np
import matplotlib.pyplot as plt
from pyspark.ml.feature import StopWordsRemover

In [4]:
startt = datetime.datetime.now()
print(startt)

2023-12-31 19:56:57.484539


### Preprocessing
This section we try to normalize news and remove useless characters (for example /n and /u200c and ...). Also find and remove stop words.

In [5]:
"""
TODO
Step 1: Remove useless charachters.
Step 2: Find and Remove stop words.
"""
# Step 1: Get 'uid', 'body' and 'title' fields of news and remove useless characters form 'body'
clean_news_rdd = news_rdd.map(lambda x:  (json.loads(x)["uid"], re.sub('[^\w\s]+', '', json.loads(x)["body"]), json.loads(x)["title"]))

# source: https://aftab.cc/article/1232-لیستی-از-کلمات-بازدارنده-در-زبان-فارسی---Stop-Words-in-Persian
stop_words = set(" گردد نحوه  دوباره‌ای خواهیمكرد توانند می‌توانند می فرماید می‌فرماید وجود دارد باشیم بنابراین برخوردار نمیمونید ساخته‌ام میخواستن خواستن خواستند خواستیم نمیگیرند همچنانكه نمیمونید میخواهند تواند باشند می‌تواند می‌باشند استفاده  جلوگیری نمیكنند رابسیار مینوشند نمیاورم رفته رفته‌ام اینطوری شده‌اند می شوند می کنند می توان می باشد می‌شوند می‌کنند می‌توان می‌باشد کند هدف کلی چنین همچنین  درباره بسیاری اساس براساس خواهند توانند اولین دومین آخرین نهایی نخستین تاكنون برداری بهترین بیشتری داشتند نخواهد دیگران همچنان ندارند روز روزهای یكدیگر همواره گذاشته  نداشته خواهیم نخواهیم درون بیرونِ پایین پاعینِ نزدیكِ دنبالِ دنبال برابر برابرِ مانند مانندِ هنگامِ هنگام عنوانِ عنوان البتّه البته وقتیكه هنگامی میكنند همانند پیششون میباشد میگردد درباره میدهیم بینمان مهمتر مهمتره بیاریم نمیبرد نمیروند نمیروم شناسید بریزید میكنیم علارغم نمیكنن اینجور میگیره میكنند میشوند می شود می کند می‌شود می‌کند می آید می‌آید این ها این جا اینجا اینها این‌ها نخواهد نخواهند خواهند خواهد هستند داشته کردیم نکردیم نکرد نکردند كردند دارند بیشتر بسیار گرفته تواند اینكه مختلف گزاری گذاری بودند دیروز امروز باشند ندارد دیگری اكنون البته مقابل امسال  قرار پارسال مانند جریان ساخته نزدیك دانستند ندانست دانست تمامی ایشان ندادند دادند هنگام نباید نداریم دارند ندارند داریم چگونه خواست هستند هست هستید هستیم سراسر گروهی آورده نماید باشی باشم باشند باشیم گویند شناسی نداشتن داشتن همچون نكرده دانند بالای خارجِ كنارِ کنار حدود حدودِ علت علّتِ كجاست چندین لطفا لطفاً مدتی مدت مدّتی میكنن هممون زدند نزدند زد نزد زدیم نزدیم دارای نشود نشوند بشود بشوند یگردی عمدا عمدتا آورده برای وبرای بتوان نتوان نتوانیم نتوانند ندیدی خوبی بخوبی بدهید نکنیم نكنند بوجود میكنم میکنیم میکنید میكند میكنه دانست دانستن دانستند دونید گرفتن گرفتند گرفتید بگیرد بگیرند بگیرم زدن زدم زدی زدیم زدند بزنیم بزن بزنند بزنید توانستید توانستن توانست تونست نداشتیم نداشتم نداشتند نداشتید نداشتی نداشت ندارم نداریم ندارید نداری نداره کردیم کردند کردن کردم كردید بود بودن بودند نبود نبودید نبودن نبودند بودید واقعا هستند هستیم هست هستید اینجا بهشون ایشان میكنن میکنند میکنیم میکنید میکنم میکند میکرد کرد کردم میکردم میرم میروم اونجا میدهم میداد میدهد كردیم می شد می‌شد میشد  میان حالی درحال درحالی سر سایر خصوص میتواند کل جز میشود کرده  کامل برخی غیر شوند میدهد بی دادند پی پس افزود وجود پیش بین وجود گرفته داشته ضمن خوب دهند رسیده میشوند رسید چون چرا برخی بودن من تو بدون طی ها نه کننده چند حتی گرفت بودند باشند هستند هستید هست هستیم چه بوده دارند مگر فقط کسی همانطور همچنان میکند نمیشود میشد اوست نگه هم آنچنان باشیم باشید یکی روی اینطور باشم باشد ادامه میخوانید میکنم کنید کنیم کنند کند کرد کردم کردن کردند داد هرچه همه هر اما های خواهد داد دیگر نیز می بود هم بر تا شد  خود حال اظهار بیان اشاره خودش خودم خودمان هست داشت کرد شما آنها این آن را با تا برای و در به از که  نسبت ترتیب  باهم نمیآید شماست آنهاست ماست برایمان برایشان برایتان بتوانیم نتوانیم بتوان نتوان اینکه  یا او آن ها آنها آنجا آن جا آن‌ها NULL برای كنند دارد كرده باید مورد آنها دیگر مردم صورت نیست است و از با در باید وی او ما شما آنها من بنده داشت داده بوده همین شوند هایی جدید كردن كنیم نشان توسط چنین برخی گیرد گفته آنان گرفت دهند بدون تمام شدند ترین قابل گوید همان آمده طریق گیری هنوز كنید سازی بلكه بودن وقتی یابد تازه آورد آنچه نخست نشده شاید زیرا ریزی گردد یک كمتر دادن ناشی چیزی آنكه بالا بعضی نبود دیده بندی نوعی آنجا كردم  رفته زیاد خویش نظیر باره جایی زیرِ زیر جلوی پیشِ پیش عقب عقبِ وسطِ وسط نزدِ نزد طبقِ طبق ضد ضدِّ مثلِ مثل بارة اثرِ اثر تول تولِ سمتِ سمت قصد قصدِ چیست چطور كدام بعری هرگز تنها خیاه مرسی كنند خودی وگرد بازم رویم ابدا میگن پراز هشان نریم واسش خاست هایت بشید دچار داره بعدا گرچه میرم باشه شدید نشون رسید دارم بیاد بودم هستن میشه برید بریم وارد یعنی اینا بدان نباش نداد میرن شدیم خودم اونا خیلی نكنه باشم کردن 0000 این است های شود شده خود كرد كند بود گفت نیز اما اند نمی بین پیش اگر همه یکی دهد داد راه سوی روی اول نام هیچ چند بیش شدن حتی ولی دوم بعد شما بار طور چون قبل تحت جای چرا سوم كنم زیر ضمن فقط آید فكر پنج كسی رفت مثل آمد ایم غیر وگو فوق سعی سپس شان هفت سری توی روب جدا هست كجا كَی آیا مگر چیز بله بلی آره آری مان تان كنن ونن مند آخه ماا بشم كنه كنی تون بهم هاش بشی ازش روى بشن بری الی كلی جور اون چیه توش بدم مون  كنن 000 های در به از که کن می را با آن یک ها شد ای تا بر وی هم ما یا سه دو هر او پس بی من چه نه طی كل تر حق اش ام ده شش جز كی یك یک جا كس ور تو را چی گه یه شو رو اه ات  چهار نا 00 ره عج ی و ع ص ۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹ ۰ a b c d e t o p x y z".split())
print(f'number of stop words: {len(stop_words)}')
# find and remove stopwords.
processed_news_rdd = clean_news_rdd.map(lambda x: (x[0], " ".join([word for word in x[1].split() if word not in stop_words]), x[2]))


number of stop words: 655


### Define the functions required to perform Min-Hashing and LSH

In [6]:
def getShingleID(shingle):
    '''This function convert shingle to an integer (id of shingle)'''
    out = 0
    # log2(p) is the number of bits for represent each char
    p = 1<<11
    for ch in shingle:
        out *= p
        out += ord(ch)%p
    return out

def get_shingles(txt):
    '''This function return the set of all k-shingles of the txt as a list of its shingles' ID'''
    shingles = set()
    for i in range(0, len(txt)-get_shingles.k+1):
        shingles.add(getShingleID(txt[i:i+get_shingles.k]))
    return list(shingles)

def my_hash(x, i, N):
    '''
        random_a and random_b are a list of random integers
        p is a prime number that bigger than 32^7-1
    '''
    return ((my_hash.random_a[i]*x+my_hash.random_b[i])%my_hash.p)%N 

def signature(shingle_list):
    '''
    This function performs Min-Hashing on shingle_list like 
    One-pass implementation which is mentioned in the slides
    and return signature vector (as a list)
    M is length of the vector that equals to r*b
    BSI is the Biggest Shingle ID 
    '''
    sig = [signature.BSI+1 for _ in range(signature.M)]
    
    for j in shingle_list:
        for i in range(signature.M):
            k = my_hash(j, i, signature.BSI)
            if k < sig[i]:
                sig[i] = k
    return sig

def bands_hashing(x): 
    '''This function hashes r numbers into one big number without losing data'''
    out = 0
    for i in range(int(lsh.r)):
        out *= bands_hashing.BSI
        out += x[i]%bands_hashing.BSI
    return out

def lsh(sig):
    '''This function drops each signature(sig) into b buckets'''
    return [bands_hashing(sig[i*lsh.r : (i+1)*lsh.r]) for i in range(lsh.b)]
    

### Define parmeters

In [7]:
N = processed_news_rdd.count()
print(f'Number of news: {N}')

k=7  # k is the size of each shingle
get_shingles.k = k
signature.BSI = 32**k-1 # BSI is the Biggest Shingle ID 
bands_hashing.BSI = 32**k-1 
print(f'Size of shingles: {k}')

r = 3 # Number of rows in Signature matrix
b = 15 # Number of bands in Signature matrix
M = r * b # Signature size for each news
lsh.r = r
lsh.b = b
signature.M = M
print(f'Divide signature matrix into <{b}> bands of <{r}> rows')


random.seed(42)
my_hash.random_a = [random.randint(100, 400) for i in range(M)]
my_hash.random_b = [random.randint(1, 200) for i in range(M)]
my_hash.p = 34359738421
print('\nlist of random integers for a and b in my hashing formula(ax+b):')
print('a: ', my_hash.random_a)
print('\nb: ', my_hash.random_b)

Number of news: 627164
Size of shingles: 7
Divide signature matrix into <15> bands of <3> rows

list of random integers for a and b in my hashing formula(ax+b):
a:  [157, 112, 240, 225, 214, 171, 152, 379, 144, 316, 116, 115, 147, 211, 219, 358, 113, 387, 201, 379, 314, 212, 329, 242, 103, 181, 316, 274, 242, 179, 210, 272, 152, 147, 294, 149, 283, 276, 235, 122, 335, 374, 163, 293, 140]

b:  [142, 76, 161, 159, 93, 148, 50, 181, 18, 12, 170, 59, 198, 75, 21, 60, 26, 98, 72, 117, 163, 94, 42, 95, 91, 54, 172, 69, 180, 175, 166, 19, 156, 163, 44, 137, 187, 63, 42, 119, 98, 70, 164, 177, 143]


### Dropping news in buckets

In [9]:
# Get the shingles of every news
news_shingles_rdd = processed_news_rdd.map(lambda x: ((x[0], x[2]), get_shingles(x[1]))) # ((news_uid, title), [shingle_id])

# Get the buckets of every news
news_buckets_rdd = news_shingles_rdd.map(lambda x: (x[0], lsh(signature(x[1])))) # ((news_uid, title), buckets)

# collect news_buckets from rdd
news_buckets = news_buckets_rdd.collect() # a list of ((news_uid, title), buckets)


### Define the functions required to show similar condidate and similar news

In [37]:
def getSimilarCandidateID(q_uid):
    '''
    This function gets uid of query news and
    return all candidate news
    q_uid: uid of query news
    '''
    # buckets of query news
    q_buckets = []
    for i in range(N):
        # news_buckets is a list of ((news_uid, title), buckets)
        if q_uid == news_buckets[i][0][0]:
            q_buckets = news_buckets[i][1]
            break
            
    similar_uid = []
    for candidate in news_buckets: # each candidate contains ((news_uid, title), buckets)
        for e in range(len(q_buckets)):
            if q_buckets[e] == candidate[1][e]:
                similar_uid.append(candidate[0][0]) # uid
                break

    return similar_uid

def getSimilar(cns, q_uid, threshold=0.4):
    ''' 
    This function get shingles of candidate news, calculate similarity
    of candidate news with query news and it returns news whose ratio 
    of common shingles with query news to the union of their shingles is 
    greater than threshold.

    cns: candidate news' shingles is a list of ((news_uid, title), [shingle_id])
    q_uid: uid of query news
    '''
    q_shingles = []
    for i in range(len(cns)):
        if q_uid == cns[i][0][0]:
            q_shingles = list(cns[i][1])
        break
        
    similar = []
    for candidate in cns:
        # s is Jaccard similarity of news = (query_news ∩ candidate_news)/union(query_news U candidate_news)
        s = len(set(candidate[1])&set(q_shingles))/len(set(candidate[1])|set(q_shingles))
        if s >= threshold:    
            similar.append((candidate[0][0], candidate[0][1], s)) # a list of (uid, title, percent)

    return similar


### Show similarity candidates for 10 news item

In [47]:
for i in range(0,10):
    # news_buckets is a list of ((news_uid, title), buckets)
    # q_buckets is buckets of query news
    q_buckets = news_buckets[i][1]
    similar = []
    for candidate in news_buckets: # each candidate contains ((news_uid, title), buckets)
        b = 0 # Number of equal buckets between query news and candidate news  
        for e in range(len(q_buckets)):
            if q_buckets[e] == candidate[1][e]:
                b +=1
        if b > 0:
            similar.append((candidate[0][1], b))
                
    if len(similar) > 1:
        print(f'\t\t+---------------+')
        print(f'\t\t|Query news(i={i})|')
        print(f'\t\t+---------------+')
        print(news_buckets[i][0][1],'\n')

        print(f'\t\t+---------------------------+')
        print(f'\t\t|Candidate news ({len(similar):3} items) |')
        print(f'\t\t+---------------------------+')
        for n in similar:
            print(n[0])
            print(f'Similar in {n[1]} bucket(s) of {lsh.b} bukets\n')
    print('---------------')

		+---------------+
		|Query news(i=0)|
		+---------------+
نوشته خاص تارتار برای لحظه جنجالی (عکس) 

		+---------------------------+
		|Candidate news (  2 items) |
		+---------------------------+
نوشته خاص تارتار برای لحظه جنجالی (عکس)
Similar in 15 bucket(s) of 15 bukets

نوشته خاص تارتار برای لحظه جنجالی (عکس)
Similar in 15 bucket(s) of 15 bukets

---------------
		+---------------+
		|Query news(i=1)|
		+---------------+
برنامه و بودجه با "همسان‌سازی حقوق بازنشستگان" موافقت کرد 

		+---------------------------+
		|Candidate news (102 items) |
		+---------------------------+
برنامه و بودجه با "همسان‌سازی حقوق بازنشستگان" موافقت کرد
Similar in 15 bucket(s) of 15 bukets

لزوم سرشماری نفوس کشور برای پیش بینی وضعیت سالمندی
Similar in 1 bucket(s) of 15 bukets

خبر جدید از همسان‌سازی حقوق بازنشستگان | جمعه بندی جلسه ۳ ساعته رئیس مجلس و رئیس سازمان برنامه بودجه
Similar in 2 bucket(s) of 15 bukets

خبر جدید از همسان‌سازی حقوق بازنشستگان | خبر خوش در راه است؟
Similar in 2 bucket(s) of 15 bu

### show similar news for «برنامه و بودجه با "همسان‌سازی حقوق بازنشستگان" موافقت کرد»

In [39]:
# news_buckets is a list of ((news_uid, title), buckets)
q_uid = news_buckets[1][0][0]
similar_news_uids_1 = getSimilarCandidateID(q_uid)

# select condidate news and do proses again on it
candidate_news_shingles_rdd = news_rdd.filter(lambda x: json.loads(x)["uid"] in similar_news_uids_1)\
                         .map(lambda x:  (json.loads(x)["uid"], re.sub('[^\w\s]+', '', json.loads(x)["body"]), json.loads(x)["title"]))\
                         .map(lambda x: (x[0], " ".join([word for word in x[1].split() if word not in stop_words]), x[2]))\
                         .map(lambda x: ((x[0], x[2]), get_shingles(x[1]))) # ((news_uid, title), [shingle_id])
                              
candidate_news_shingles = candidate_news_shingles_rdd.collect()


similar = getSimilar(candidate_news_shingles, q_uid)

print(f'***{len(similar)} similar news were found ***\n')
similar.sort(key=lambda x: -x[2])
for uid, title, percent in similar:
    print(title)
    print(f'Jaccard similariy = {percent:.2f}', end='\n\n')

***28 similar news were found ***

برنامه و بودجه با "همسان‌سازی حقوق بازنشستگان" موافقت کرد
Jaccard similariy = 1.00

برنامه و بودجه با "همسان‌سازی حقوق بازنشستگان" موافقت کرد
Jaccard similariy = 1.00

تصمیم شبانه دولت / خبر خوب برای بازنشستگان رسید - خبرجو۲۴
Jaccard similariy = 0.82

تصمیم شبانه دولت / خبر خوب برای بازنشستگان رسید - خبرجو۲۴
Jaccard similariy = 0.82

موافقت اولیه دولت با متناسب سازی حقوق بازنشستگان لشگری و کشوری- اخبار مجلس و دولت ایران - اخبار سیاسی تسنیم | Tasnim
Jaccard similariy = 0.81

زنگنه: همسان‌سازی بازنشستگان تا پایان برنامه هفتم محقق می‌شود- اخبار مجلس و دولت ایران - اخبار سیاسی تسنیم | Tasnim
Jaccard similariy = 0.77

خبر خوب برای همسان سازی حقوق بازنشستگان | زنگنه سخنگوی کمیسیون
Jaccard similariy = 0.68

خبر خوب برای همسان سازی حقوق بازنشستگان | زنگنه سخنگوی کمیسیون
Jaccard similariy = 0.68

خبر خوش برای بازنشستگان/ همسان‌سازی تا پایان برنامه محقق می‌شود
Jaccard similariy = 0.59

متناسب‌سازی حقوق بازنشستگان تا پایان برنامه هفتم
Jaccard similariy = 0.57

ت

### show simlilar news for «خبرگزاری فارس - پیشنهاد وزیر کار برای اختصاص بخشی از کتاب‌های درسی به تکریم سالمندان»

In [41]:
# news_buckets is a list of ((news_uid, title), buckets)
q_uid = news_buckets[7][0][0]
similar_news_uids_1 = getSimilarCandidateID(q_uid)

# select condidate news and do proses again on it
candidate_news_shingles_rdd = news_rdd.filter(lambda x: json.loads(x)["uid"] in similar_news_uids_1)\
                         .map(lambda x:  (json.loads(x)["uid"], re.sub('[^\w\s]+', '', json.loads(x)["body"]), json.loads(x)["title"]))\
                         .map(lambda x: (x[0], " ".join([word for word in x[1].split() if word not in stop_words]), x[2]))\
                         .map(lambda x: ((x[0], x[2]), get_shingles(x[1]))) # ((news_uid, title), [shingle_id])
                              
candidate_news_shingles = candidate_news_shingles_rdd.collect()


similar = getSimilar(candidate_news_shingles, q_uid)

print(f'***{len(similar)} similar news were found ***\n')
similar.sort(key=lambda x: -x[2])
for uid, title, percent in similar:
    print(title)
    print(f'Jaccard similariy = {percent:.2f}', end='\n\n')

***8 similar news were found ***

خبرگزاری فارس - پیشنهاد وزیر کار برای اختصاص بخشی از کتاب‌های درسی به تکریم سالمندان
Jaccard similariy = 1.00

خبرگزاری فارس - پیشنهاد وزیر کار برای اختصاص بخشی از کتاب‌های درسی به تکریم سالمندان
Jaccard similariy = 1.00

پیشنهاد وزیر کار برای اختصاص بخشی از کتاب‌های درسی به تکریم سالمندان
Jaccard similariy = 0.84

پیشنهاد وزیر کار برای اختصاص بخشی از کتاب‌های درسی به تکریم سالمندان
Jaccard similariy = 0.84

نباید طوری کار کنیم که نگاه سالمندان به جیب دیگران باشد
Jaccard similariy = 0.67

نباید طوری کار کنیم که نگاه سالمندان به جیب دیگران باشد
Jaccard similariy = 0.67

آخرین موضع گیری صولت مرتضوی، وزیر کار درباره حقوق بازنشستگان امروز چهارشنبه 19 مهر 1402 - اندیشه معاصر
Jaccard similariy = 0.60

آخرین موضع گیری صولت مرتضوی، وزیر کار درباره حقوق بازنشستگان امروز چهارشنبه 19 مهر 1402 - اندیشه معاصر
Jaccard similariy = 0.60



### show simlilar news for «عرضه 6 خودروی وارداتی در سامانه یکپارچه آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim»

In [46]:
# news_buckets is a list of ((news_uid, title), buckets)
q_uid = news_buckets[3][0][0]
similar_news_uids_1 = getSimilarCandidateID(q_uid)

# select condidate news and do proses again on it
candidate_news_shingles_rdd = news_rdd.filter(lambda x: json.loads(x)["uid"] in similar_news_uids_1)\
                         .map(lambda x:  (json.loads(x)["uid"], re.sub('[^\w\s]+', '', json.loads(x)["body"]), json.loads(x)["title"]))\
                         .map(lambda x: (x[0], " ".join([word for word in x[1].split() if word not in stop_words]), x[2]))\
                         .map(lambda x: ((x[0], x[2]), get_shingles(x[1]))) # ((news_uid, title), [shingle_id])
                              
candidate_news_shingles = candidate_news_shingles_rdd.collect()


similar = getSimilar(candidate_news_shingles, q_uid)

print(f'***{len(similar)} similar news were found ***\n')
similar.sort(key=lambda x: -x[2])
for uid, title, percent in similar:
    print(title)
    print(f'Jaccard similariy = {percent:.2f}', end='\n\n')

***22 similar news were found ***

عرضه 6 خودروی وارداتی در سامانه یکپارچه آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim
Jaccard similariy = 1.00

عرضه 6 خودروی وارداتی در سامانه یکپارچه آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim
Jaccard similariy = 1.00

 عرضه ۶ خودروی جدید در سامانه یکپارچه
Jaccard similariy = 0.80

 عرضه ۶ خودروی جدید در سامانه یکپارچه
Jaccard similariy = 0.80

 سامانه یکپارچه فروش با این خودروها باز شد 
Jaccard similariy = 0.67

 سامانه یکپارچه فروش با این خودروها باز شد 
Jaccard similariy = 0.67

عرضه 6 خودروی وارداتی آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim
Jaccard similariy = 0.67

عرضه 6 خودروی وارداتی آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim
Jaccard similariy = 0.67

سامانه یکپارچه فروش برای خودروهای وارداتی باز شد
Jaccard similariy = 0.66

سامانه یکپارچه فروش برای خودروهای وارداتی باز شد
Jaccard similariy = 0.66

جزئیات عرضه ۶ خودروی وارداتی در سامانه یکپارچه | زمان ثبت نام و اسامی خودروها
Jacca

In [379]:
for i in range(1,100):
    # news_buckets is a list of ((news_uid, title), buckets)
    q_buckets = news_buckets[i][1]
    cnt = 0
    similar = set()
    for bucket in news_buckets:
        for e in range(len(q_buckets)):
            if q_buckets[e] == bucket[1][e]:
                similar.add(bucket[0][1])
                # if cnt > 0:
                #     if(cnt == 1):
                #         print(news_buckets[i][0][1])
                #     print(bucket[0][1])
                cnt += 1
                break
    if len(similar) > 1:
        for n in similar:
            print(n)
    print('---------------')

---------------
---------------
---------------
---------------
---------------
---------------
---------------
۹ عضو هیأت علمی دانشگاه آزاد اسلامی به مرتبه استادی ارتقا یافتند
۹ عضو هیات علمی دانشگاه آزاد اسلامی به مرتبه استادی ارتقا یافتند
---------------
---------------
پرسپولیس حکم کمیته انضباطی را نپذیرفت
پرسپولیس حکم کمیته انضباطی را رد کرد
---------------
---------------
---------------
قیمت خودرو‌های سایپا امروز شنبه ۱۳ آبان ۱۴۰۲+ جدول
اخبار لحظه‌ای از حضور تیم النصر و ستارگانش در ایران
وعده‌هایی که برای بازنشستگان به نتیجه نرسید
رشد قیمت طلا
درخواست فوری تهران در واکنش به تهدید وحشیانه مقام صهیونسیتی
اعضای هیات رئیسه خانه احزاب انتخاب شدند
قیمت خودروهای سایپا امروز پنجشنبه ۲۰ مهر ۱۴۰۲+ جدول
حقوق کارمندان و کارگران امسال هم مثل پارسال
توقف پذیرش و تبادل چک‌های صیادی قدیمی در سامانه چکاوک از اول مهر
کیفیت هوای تهران امروز قابل قبول است
ترکیب زنان فوتبالیست ایران برای تاریخ‌سازی
لحظه معرفی و اهدای توپ طلا به لیونل مسی+ فیلم
رشد 58 درصدی پرداخت وام قرض‌الحسنه فرزندآوری در بانک آین

In [385]:
candidate_pairs2 = buckets

In [388]:
minb = 8
for i in range(1,100):
    # news_buckets is a list of ((news_uid, title), buckets)
    q_buckets = news_buckets[i][1]
    cnt = 0
    similar = set()
    for bucket in news_buckets:
        b = 0
        for e in range(len(q_buckets)):
            if q_buckets[e] == bucket[1][e]:
                b +=1
        if b > 0:
            similar.add((bucket[0][1], b))
            cnt += 1
                
    if len(similar) > 1:
        print(news_buckets[i][0][1])
        print("")
        
        for n in similar:
            if n[1] > minb or 1:
                print(n[0])
                print(n[1])
    print('---------------')

عرضه 6 خودروی وارداتی در سامانه یکپارچه آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim

عرضه 6 خودروی وارداتی در سامانه یکپارچه آغاز شد- اخبار صنعت و تجارت - اخبار اقتصادی تسنیم | Tasnim
20
آغاز عرضۀ خودروهای وارداتی از چهارشنبه
2
---------------
---------------
---------------
شش عادت غذایی برای سوزاندن چربی شکم

با این علائم فلج معده را جدی بگیرید
1
شش عادت غذایی برای سوزاندن چربی شکم
20
---------------
درگیری ارتش با طالبان پاکستان در مرز افغانستان | ارتش پاکستان

خبرگزاری فارس - درگیری ارتش با طالبان پاکستان  در مرز افغانستان
1
درگیری ارتش با طالبان پاکستان در مرز افغانستان | ارتش پاکستان
20
---------------
---------------
---------------
۹ عضو هیات علمی دانشگاه آزاد اسلامی به مرتبه استادی ارتقا یافتند

۹ عضو هیات علمی دانشگاه آزاد اسلامی به مرتبه استادی ارتقا یافتند
20
۹ عضو هیأت علمی دانشگاه آزاد اسلامی به مرتبه استادی ارتقا یافتند
13
---------------
خبر مهم| پیش‌ بینی عجیب درباره قیمت مسکن - اندیشه معاصر

خبر تلخ برای بازنشستگان| شوک دیگری به بازنشستگان عزیز؟ - اندیش

In [368]:
# sh1 = signature_rdd.take(10)
# news_buckets = lsh_rdd.collect()

2023-12-29 20:11:47.139212


In [376]:
endt = datetime.datetime.now()
print(endt)

2023-12-29 20:53:13.965212


In [174]:
# x = ((1,2),)
y = ((2,2),)
x = x + y
x

((1, 2), (2, 2), (2, 2), (2, 2), (2, 2))

In [ ]:
def isprime(num):
    flag = False

    # check for factors
    for i in range(2, num):
        if (num % i) == 0:
            # if factor is found, set flag to True
            flag = True
            # break out of loop
            break

    # check if flag is True
    return not flag


In [112]:
def get_shingles(txt, k):
    shingles = set()
    for i in range(0, len(txt)-k+1):
        shingles.add(txt[i:i+k])
    return shingles


def myfunc2(t):
    lis = []
    for n in t[1]:
        lis.append((n, (t[0],)))
    return lis

# befor run this part, you most run 5 first part in HW1 
k=6
shingles_uid_rdd = processed_news_rdd.map(lambda x: (get_shingles2(x[1], k), x[0], x[2])) # ([shingles], news_uid, title)

shingle_uid_rdd = shingles_uid_rdd.flatMap(lambda x: ((s, ((x[1], x[2]),)) for s in x[0])) # (shingle, (news_uid, title))

shingle_uids_rdd = shingle_uid_rdd.reduceByKey(lambda x, y: x + y) # (shingle, ((news_uid, title)))

shingle_uids_filtered_rdd = shingle_uids_rdd.filter(lambda x: len(x)<N*0.7) # (shingle, (news_uids))

uid_shingles_rdd = shingle_uids_filtered_rdd.flatMap(lambda x: myfunc2((getShingleID(x[0]), x[1]))) # ((news_uid, title), shingle_id)

news_shingles_rdd = uid_shingles_rdd.reduceByKey(lambda x, y: x + y) # ((news_uid, title), (shingle_id))
# now we have Shingles x Documents
r = 5
b = 120
M = r * b
signature_rdd = news_shingles_rdd.map(lambda x: (x[0], signature(x[1]))) # ((news_uid, title), signature)

lsh_rdd = signature_rdd.map(lambda x: (x[0], lsh(x[1]))) # ((news_uid, title), lsh)

